# Study the correlation between the SSP Parameters

Generate simulation data to study the correlation between the parameters using `fors2tostellarpopsynthesis`package

- Author Sylvie Dagoret-Campagne
- Afflilation : IJCLab/IN2P3/CNRS
- Organisation : LSST-DESC
- creation date : 2023-12-05
- last update : 2023-12-05


| computer | processor | kernel              |    date     |
| --- | --- | --- | --- |
| CC       | CPU       | conda_jax0325_py310 | 2023-11-10  |
| macbookpro | CPU | conda_jaxcpu_dsps_py310 | 2023-11-10  | 


libraries 
=========

jax
---

- jaxlib-0.3.25+cuda11.cudnn82
- jaxopt
- optax
- corner
- arviz
- numpyro
- graphviz

sps
---

- fsps
- prospect
- dsps
- diffstar
- diffmah



(conda_jax0325_py310) 
`/pbs/throng/lsst/users/dagoret/desc/StellarPopulationSynthesis>pip list | grep` 

| lib | version |
|--- | --- | 
|jax  |                         0.4.20 |
|jaxlib |                       0.4.20 |
|jaxopt  |                      0.8.2 |



## examples

- jaxcosmo : https://github.com/DifferentiableUniverseInitiative/jax-cosmo-paper/blob/master/notebooks/VectorizedNumPyro.ipynb
- on atmosphere : https://github.com/sylvielsstfr/FitDiffAtmo/blob/main/docs/notebooks/fitdiffatmo/test_numpyro_orderedict_diffatmemul_5params_P_pwv_oz_tau_beta.ipynb

## Import

### import external packages

In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
import re
import pickle 
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.cm as cmx
import collections
from collections import OrderedDict
import re
import matplotlib.gridspec as gridspec
from sklearn.gaussian_process import GaussianProcessRegressor, kernels

In [ ]:
import jax
import jax.numpy as jnp
from jax import vmap
import jaxopt
import optax
jax.config.update("jax_enable_x64", True)
from interpax import interp1d

from jax.lax import fori_loop
from jax.lax import select,cond
from jax.lax import concatenate

In [ ]:
import numpyro
from numpyro import optim
from numpyro.diagnostics import print_summary
import numpyro.distributions as dist
from numpyro.distributions import constraints
from numpyro.infer import MCMC, HMC, NUTS, SVI, Trace_ELBO
from numpyro.infer.autoguide import AutoBNAFNormal, AutoMultivariateNormal
from numpyro.infer.reparam import NeuTraReparam
from numpyro.handlers import seed, trace, condition


import corner
import arviz as az

In [ ]:
import pickle

### import internal packages

In [ ]:
from fors2tostellarpopsynthesis.parameters  import SSPParametersFit,paramslist_to_dict

In [ ]:
from fors2tostellarpopsynthesis.fitters.fitter_jaxopt import (SSP_DATA,mean_spectrum,mean_mags,mean_sfr,ssp_spectrum_fromparam)

In [ ]:
from fors2tostellarpopsynthesis.fitters.fitter_numpyro import(PARAM_SIMLAW_NODUST,PARAM_SIMLAW_WITHDUST,
                            PARAM_NAMES,PARAM_VAL,PARAM_MIN,PARAM_MAX,PARAM_SIGMA)

from fors2tostellarpopsynthesis.fitters.fitter_numpyro import(galaxymodel_nodust_av,galaxymodel_nodust,galaxymodel_withdust_av,galaxymodel_withdust)

In [ ]:
from fors2tostellarpopsynthesis.fitters.fitter_util import plot_params_kde

## Configuration

### matplotlib configuration

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  12
plt.rcParams['font.size'] = 12

## Fit parameters

In [ ]:
p = SSPParametersFit()

In [ ]:
#p.INIT_PARAMS = p.INIT_PARAMS.at[-4].set(0.)
#p.INIT_PARAMS = p.INIT_PARAMS.at[-3].set(0.)
#p.INIT_PARAMS = p.INIT_PARAMS.at[-2].set(0.)
#p.INIT_PARAMS = p.INIT_PARAMS.at[-1].set(1.)

In [ ]:
wlsall,spec_rest,spec_rest_att = ssp_spectrum_fromparam(p.DICT_PARAMS_true,0)

In [ ]:
df_params = pd.DataFrame({"name":PARAM_NAMES,"min": PARAM_MIN,"val": PARAM_VAL,"max": PARAM_MAX,"sig":PARAM_SIGMA})

In [ ]:
df_params  = df_params.round(decimals=2)
df_params

In [ ]:
print(PARAM_SIMLAW_NODUST)  
print(PARAM_SIMLAW_WITHDUST) 
print(PARAM_NAMES)
print(PARAM_VAL)
print(PARAM_MIN) 
print(PARAM_MAX)
print(PARAM_SIGMA)

In [ ]:
print((PARAM_VAL-PARAM_MIN)/PARAM_SIGMA)

In [ ]:
print((PARAM_MAX-PARAM_VAL)/PARAM_SIGMA)

In [ ]:
z_obs = 0.5
sigmarel_obs = 0.1
sigma_obs = 1e-11

In [ ]:
with seed(rng_seed=42):
    spec_nodust,sigmanodust_obs = galaxymodel_nodust_av(wlsall,Fobs=None,
                       initparamval = PARAM_VAL, minparamval = PARAM_MIN,maxparamval = PARAM_MAX,
                       sigmaparamval = PARAM_SIGMA,paramnames = PARAM_NAMES,z_obs= z_obs, sigmarel = sigmarel_obs)

In [ ]:
with seed(rng_seed=42):
    spec_withdust,sigmadust_obs = galaxymodel_withdust_av(wlsall,Fobs=None,
                       initparamval = PARAM_VAL, minparamval = PARAM_MIN,maxparamval = PARAM_MAX,
                       sigmaparamval = PARAM_SIGMA,paramnames = PARAM_NAMES,z_obs= z_obs, sigmarel = sigmarel_obs)

In [ ]:
fig = plt.figure(figsize=(10,5))
gs = fig.add_gridspec(2, 1,height_ratios=[3.,1.])
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.errorbar(wlsall,spec_nodust,yerr=sigmanodust_obs,fmt='o',ms=0.5 ,linewidth=2, capsize=0, c='k', label="no dust")
ax1.errorbar(wlsall,spec_withdust,yerr=sigmadust_obs, fmt='o', ms=0.5,linewidth=2, capsize=0, c='r', label="with dust")
ax1.set_xlabel("$\lambda$ (nm)")
ax1.set_ylabel("DSPS SED ")
ax1.legend()
ymin = 1e-11
ymax = jnp.max(spec_nodust)*2.
ax1.set_ylim(ymin,ymax)
ax1.set_xlim(1e2,1e6)
ax1.grid();

#error
ax2.set_yscale('log')
ax2.set_xscale('log')

ax2.plot(wlsall,sigmanodust_obs,'k-')
ax2.plot(wlsall,sigmadust_obs,'r-')
ax2.set_xlabel("$\lambda$ (nm)")
ax2.set_ylabel("DSPS SED error")

ymin = 1e-11
ymax = jnp.max(sigmanodust_obs)*5.
ax2.set_ylim(ymin,ymax)
ax2.set_xlim(1e2,1e6)
ax2.grid();

plt.tight_layout()

## Bayesian modelling

In [ ]:
condlist_fix = jnp.where(PARAM_SIMLAW_NODUST == "fixed",True,False)
condlist_fix

In [ ]:
condlist_uniform = jnp.where(PARAM_SIMLAW_NODUST == "uniform",True,False)
condlist_uniform

In [ ]:
numpyro.render_model(galaxymodel_nodust, model_args=(jnp.array([0.]),jnp.array([1.]),
                                                     PARAM_VAL, 
                                                     PARAM_MIN,
                                                     PARAM_MAX,
                                                     PARAM_SIGMA, 
                                                     PARAM_NAMES,
                                                     z_obs,sigmanodust_obs),render_distributions=True)

In [ ]:
numpyro.render_model(galaxymodel_withdust, model_args=(jnp.array([0.]),jnp.array([1.]),
                                                     PARAM_VAL, 
                                                     PARAM_MIN,
                                                     PARAM_MAX,
                                                     PARAM_SIGMA, 
                                                     PARAM_NAMES,
                                                     z_obs,sigma_obs),render_distributions=True)

In [ ]:
# So, let's generate the data at the fiducial parameters
fiducial_model = condition(galaxymodel_nodust, p.DICT_PARAMS_true)
# Run the conditionned model (if not conditionned the priors are executed)
trace_data_nodust = trace(seed(fiducial_model, jax.random.PRNGKey(42))).get_trace(wlsall,
                       initparamval = PARAM_VAL,                                                           
                       minparamval = PARAM_MIN,
                       maxparamval = PARAM_MAX,
                       sigmaparamval = PARAM_SIGMA,
                       paramnames = PARAM_NAMES,
                       z_obs = z_obs,
                       sigma = sigmanodust_obs)

In [ ]:
spec_nodust = trace_data_nodust['F']["value"]

In [ ]:
# So, let's generate the data at the fiducial parameters
fiducial_model = condition(galaxymodel_withdust, p.DICT_PARAMS_true)
# Run the conditionned model (if not conditionned the priors are executed)
trace_data_withdust = trace(seed(fiducial_model, jax.random.PRNGKey(42))).get_trace(wlsall,
                       initparamval = PARAM_VAL,                                                             
                       minparamval = PARAM_MIN,
                       maxparamval = PARAM_MAX,
                       sigmaparamval = PARAM_SIGMA,
                       paramnames = PARAM_NAMES,
                       z_obs = z_obs,
                       sigma = sigmadust_obs)

In [ ]:
#trace_data_withdust

In [ ]:
spec_withdust = trace_data_withdust['F']["value"]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,3))
ax.errorbar(wlsall,spec_nodust,yerr=sigma_obs,fmt='o',ms=0.5 ,linewidth=2, capsize=0, c='k', label="no dust")
ax.errorbar(wlsall,spec_withdust,yerr=sigma_obs, fmt='o', ms=0.5,linewidth=2, capsize=0, c='r', label="with dust")
ax.set_xlabel("$\lambda$ (nm)")
ax.set_ylabel("DSPS spectrim")
ax.legend()
ax.set_yscale('log')
ax.set_ylim(1e-11,1e-5)
ax.set_xlim(1e2,1e6)
ax.set_xscale('log')
ax.grid();


## Info

In [ ]:
df_info = pd.Series({"z_obs":z_obs,"sigmarel_obs":sigmarel_obs})

In [ ]:
df_info

## MCMC without dust

In [ ]:
FLAG_NODUST = True

### Output file for MCMC without dust

In [ ]:
fileout_pickle = f"DSPS_nodust_mcmc_params_wide.pickle"
fileout_csv = f"DSPS_nodust_mcmc_params_wide.csv"
fileout_hdf = f"DSPS_nodust_mcmc_params_wide.hdf"

### Run MCMC

In [ ]:
if FLAG_NODUST:

    # Run NUTS.
    rng_key = jax.random.PRNGKey(42)
    rng_key, rng_key0, rng_key1, rng_key2 = jax.random.split(rng_key, 4)


    kernel = NUTS(galaxymodel_nodust, dense_mass=True, target_accept_prob=0.9,init_strategy=numpyro.infer.init_to_median())

    num_samples = 5_000
    n_chains = 4
    mcmc = MCMC(kernel, num_warmup=1_000, num_samples=num_samples,  
            num_chains=n_chains,
            chain_method='vectorized',
            progress_bar=True)
    # see https://forum.pyro.ai/t/cannot-find-valid-initial-parameters-when-using-nuts-for-simple-gaussian-mixture-model-in-numpyro/2181/2
    with numpyro.validation_enabled():
        mcmc.run(rng_key, wlsin=wlsall, Fobs=spec_nodust, initparamval = PARAM_VAL,
                       minparamval = PARAM_MIN,
                       maxparamval = PARAM_MAX,
                       sigmaparamval = PARAM_SIGMA,
                       paramnames = PARAM_NAMES,
                       z_obs = z_obs,
                       sigma = sigmanodust_obs,
                       extra_fields=('potential_energy',))
        mcmc.print_summary()
        samples_nuts = mcmc.get_samples()

### Results of MCMC no dust

In [ ]:
if FLAG_NODUST:
    az.ess(samples_nuts, relative=True)  # efficacité relative

In [ ]:
if FLAG_NODUST:
    data = az.from_numpyro(mcmc)
    az.plot_trace(data, compact=True);

In [ ]:
if FLAG_NODUST:
    data

### Pandas dataframe

In [ ]:
if FLAG_NODUST:
    df_nodust = pd.DataFrame(samples_nuts) 

In [ ]:
#df_nodust

In [ ]:
if FLAG_NODUST:
    with open(fileout_pickle, 'wb') as handle:
        pickle.dump(samples_nuts, handle, protocol=pickle.HIGHEST_PROTOCOL)

    df_nodust.to_csv(fileout_csv)

    df_nodust.to_hdf(fileout_hdf, key="dsps_mcmc_nodust",mode='a', complevel=9) 
    df_info.to_hdf(fileout_hdf,key="info",mode='a')
    df_params.to_hdf(fileout_hdf,key="params",mode='a')

## MCMC With DUST

In [ ]:
FLAG_DUST = True

### output files

In [ ]:
fileout_pickle = f"DSPS_dust_mcmc_params_wide.pickle"
fileout_csv = f"DSPS_dust_mcmc_params_wide.csv"
fileout_hdf = f"DSPS_dust_mcmc_params_wide.hdf"

### run MCMC with Dust

In [ ]:
if FLAG_DUST: 
    # Run NUTS.
    rng_key = jax.random.PRNGKey(42)
    rng_key, rng_key0, rng_key1, rng_key2 = jax.random.split(rng_key, 4)


    kernel = NUTS(galaxymodel_withdust, dense_mass=True, target_accept_prob=0.9,
              init_strategy=numpyro.infer.init_to_median())
    
    num_samples = 5_000
    n_chains = 4
    mcmc = MCMC(kernel, num_warmup=1_000, num_samples=num_samples,  
            num_chains=n_chains,
            chain_method='vectorized',
            progress_bar=True)

    # see https://forum.pyro.ai/t/cannot-find-valid-initial-parameters-when-using-nuts-for-simple-gaussian-mixture-model-in-numpyro/2181/2
    with numpyro.validation_enabled():
        mcmc.run(rng_key, wlsin=wlsall, Fobs=spec_withdust, initparamval = PARAM_VAL,
                       minparamval = PARAM_MIN,
                       maxparamval = PARAM_MAX,
                       sigmaparamval = PARAM_SIGMA,
                       paramnames = PARAM_NAMES,
                       z_obs = z_obs,
                       sigma = sigmadust_obs,
                       extra_fields=('potential_energy',))
        mcmc.print_summary()
        samples_nuts = mcmc.get_samples()

### results of MCMC with DUST

In [ ]:
az.ess(samples_nuts, relative=True)  # efficacité relative

In [ ]:
data = az.from_numpyro(mcmc)
az.plot_trace(data, compact=True);
plt.tight_layout()

### pandas dataframe with dust

In [ ]:
df_dust = pd.DataFrame(samples_nuts) 

In [ ]:
#df_dust

In [ ]:
with open(fileout_pickle, 'wb') as handle:
    pickle.dump(samples_nuts, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_dust.to_csv(fileout_csv)
df_dust.to_hdf(fileout_hdf, key="dsps_mcmc_dust",mode='a', complevel=9) 
df_info.to_hdf(fileout_hdf,key="info",mode='a')
df_params.to_hdf(fileout_hdf,key="params",mode='a')